## Demand
Let $i$ := be a variable representing the month of the year where $i \in$ {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}  
        
Let $d_{i}$:= be a variable representing the customer electricity demand for month $i$ in kilowatt-hours where $d_i \geq 0$ 

Let $h_{i}$ := be a variable representing the customer heat demand for month $i$ in BTU's where $h_i \geq 0$    

## Power Sources
Let $j$ := be a variable representing service (the source of power) where $j$ $\in$ {1, 2, 3, 4, ....}  
Let $oc_j$ := be the opening cost in US dollars up initially setting up one unit of a specific service $j$ where $oc_j \geq 0$  
Let $ic_j$ := be the incremental cost in US dollars of opening up an additional unit of a specific service $j$ where $ic_j \geq 0$   
Let $mc_j$ := be the monthly cost in US dollars of maintaining one unit of service $j$ where $mc_j \geq 0$   
Let $\rho_{i,j}$ := be the monthly $i$ capacity one unit of service $j$ in kilowatt-hours where $\rho_j \geq 0$  
Let $zc$ := be the cost of on-grid power in units of dollers per kilowatt hour

## Environmental Impact
Let $ec_j$:= be the variable representing the relative environmental impact of service $j$ measured in units _______

## Decision Variables
Let $x_j$ := be a binary variable representing whether or not the service $j$ is being chosen to use where $x_j \in$ {0,1}  
Let $y_{j}$ := be an integer variable representing the number of units of product $j$ where $y_j$ = {$y_j$:= $y_j$ is an integer, $y_j \geq$ 0}  
Let $z_i$ := be the continuous variable that represents the amount of power needed from the grid in kilowatt-hours.

## Objective Function
We want to minimize the cost
Therefore, the objective function is the sum of the three kinds of costs: a) opening cost (oc) the cost of initially setting up a new service b) monthly cost (mc) and c) environmental cost (ec)
$$\min \sum_{j} oc_jx_j + \sum_{j} ic_jy_j + \sum_{j} mc_{j}y_{j} + \sum_{j} ec_{j}y_{j}  + \sum_i z_{i}zc + \sum_i z_{i}ec_z$$

## Constraints

### Demand Constraints
1) Meet customer demand for electricity per month

$$\forall i \sum_{j\in 'elec'} y_{j}\rho_{i,j} + z_i \geq d_i$$

2) Meet customer demand for heat per month
$$\forall i \sum_{j\in 'heat'} y_{j}\rho_{i,j} + z_i \geq h_i$$

3) You have an initial budget of the amount of capital you have on hand to shell out for initial set up.
$$  \sum_{j} oc_jx_j + \sum_{j} ic_jy_j \leq $200,000 $$

4) You also have a monthly budget.
$$ \sum_{j} mc_{j}y_{j} \leq $500 $$


##### Note: only pay one opening cost per energy type installed (e.g., when one small and one large solar installed, only pay one opening cost)

In [ ]:
import gurobipy as gp

In [ ]:
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
electricity_demand = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
heat_demand = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

# Grid Energy
grid_cost_kwh = .5 # cost per kwh
grid_cost_env = 1  # environmental cost

CAPITAL_BUDGET = 200000
OPEX_BUDGET = 500

class Product:
    HEAT = 'H'
    ELEC = 'E'
    ENERGY_TYPES = ['Solar', 'Wind', 'Geothermal', 'Biomass']
    def __init__(self, name, utility_type, energy_type, opening_cost, incremental_cost, 
                 maintenance_cost, environmental_cost, monthly_capacity):
        self.name = name
        self.ut = utility_type       # H or E
        self.et = energy_type        # ENERGY_TYPES
        self.oc = opening_cost       # Dollars Per Opening
        self.ic = incremental_cost   # Dollars Per Product
        self.mc = maintenance_cost   # Annual Maintenance Cost
        self.ec = environmental_cost # Per KwH
        self.ca = monthly_capacity   # KwH per Month
        self.x = None
        self.y = None
    
    

# type, name, oc, ic, mc, ec, p
products = [
    Product('smallsolar', 'E', 'Solar', 10000, 1000, 0, 1, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
    Product('smallwind', 'E', 'Wind', 15000, 100, 0, 10, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
]

# Social Cost of Carbon
scc = 10

m = gp.Model('sustainable_house')

grid = [m.addVar(vtype=gp.GRB.INTEGER) for _ in months]
for name, product in product.items():
    product.x = m.addVar(vtype=gp.GRB.BINARY)
    product.y = m.addVar(vtype=gp.GRB.INTEGER)

m.setObjective(sum(product.oc * product.x for product in products) + 
               sum(product.y * (product.ic + product.mc + product.ec) for product in products) + 
               sum(monthly_grid * (grid_cost_kwh + grid_cost_env) for monthly_grid in grid), gp.GRB.MINIMIZE)

for i in months:
    m.addConstr(sum(product.ca[i] * product.y for product in products if product.et == Product.ELEC) + grid[i] >= electricity_demand[i])
    m.addConstr(sum(product.ca[i] * product.y for product in products if product.et == Product.HEAT) + grid[i] >= heat_demand[i])

for product in products:
    m.addConstr(sum(product.x for product in products if product.et == et) * M >= product.y)
    
for et in Product.ENERGY_TYPES:
    m.addConstr(sum(product.x for product in products if product.et == et) <= 1)

m.addConstr(sum(product.oc * product.x + product.ic * product.y for product in products) <= CAPITAL_BUDGET)
m.addConstr(sum(product.mc * product.y for product in products) <= OPEX_BUDGET)